## Problem 2

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.optim as optim
import random
import math
import time
import func3

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [3]:
def seqGen(n):
    ind = random.sample(range(10), 5)
    ind.sort()
    randmu = 2*torch.rand((5)) - 1
    T = random.randint(0, n)
    haspoint = random.random()
    seq = torch.randn((n,10))
    if haspoint < 0.8:
        for t in range(n):
                
                j = 0
                for i in range(10):
                    if i in ind:
                        seq[t][i] = seq[t][i] + randmu[j]
                        j = j+1
    else: 
    else
    return seq, ind, randmu, haspoint, T

In [7]:
seq, ind, randmu, haspoint, T = seqGen(50)
print(len(seq))
print(ind)
print(randmu)
print(haspoint, T)
for i in range(len(seq)):
    print(i, seq[i])

50
[0, 2, 4, 6, 8]
tensor([ 0.9979,  0.3047, -0.1415, -0.7205,  0.8178])
0 22
0 tensor([ 1.0399,  1.1474, -0.9378, -0.3513,  2.3777,  0.2853,  0.2406,  1.6800,
         1.3886, -0.0181])
1 tensor([ 0.7223,  0.6952,  1.0068,  0.1953, -0.1839,  1.2503,  1.4580, -0.5539,
        -0.1916,  0.7588])
2 tensor([ 0.6995,  0.1181, -0.2898,  0.8608, -2.6839, -0.1904,  0.7820, -1.0848,
        -2.1385,  2.4629])
3 tensor([-0.4236,  1.2407,  0.7797,  0.4725,  0.1904, -1.2370, -1.0132, -1.8932,
        -0.2487, -0.7997])
4 tensor([ 0.3581, -1.5211,  0.0377, -0.2268,  0.2915, -0.2531,  0.5637,  2.4980,
         0.6208,  1.2561])
5 tensor([-0.6611,  0.0340,  1.2407, -0.6805, -1.2731,  0.7915,  1.3916, -0.2622,
        -0.7772, -0.0042])
6 tensor([ 0.0613,  0.0961,  0.4298,  1.2918,  1.7247,  0.7117, -1.7152, -0.4974,
         1.5140,  0.1750])
7 tensor([-0.5678, -0.5550, -1.6377, -1.2251, -1.1410, -0.7413,  1.4224,  1.7350,
         1.0121, -0.6490])
8 tensor([ 0.4246,  1.7240,  0.1867, -1.0061,  0.8

In [403]:
class changeDetectRNN(nn.Module):
    def __init__(self):
        super(changeDetectRNN, self).__init__()
        self.current_itemf = torch.nn.Linear( in_features = 10, out_features = 100, bias = True )
        self.current_itemb = torch.nn.Linear( in_features = 10, out_features = 100, bias = True )

        self.prev_itemf1 = torch.nn.Linear( in_features = 100, out_features = 100, bias = False )
        self.prev_itemb1 = torch.nn.Linear( in_features = 100, out_features = 100, bias = False )

        self.prev_itemf2 = torch.nn.Linear( in_features = 100, out_features = 100, bias = False )
        self.prev_itemb2 = torch.nn.Linear( in_features = 100, out_features = 100, bias = False )

        self.prev_itemf3 = torch.nn.Linear( in_features = 100, out_features = 100, bias = False )
        self.prev_itemb3 = torch.nn.Linear( in_features = 100, out_features = 100, bias = False )

        
        self.normf = nn.LayerNorm(100)
        self.normb = nn.LayerNorm(100)
        
        self.point_detector = nn.Sequential(nn.Linear(200,1000), nn.LayerNorm(1000),nn.Sigmoid(),nn.Linear(1000,1000),nn.Sigmoid(), nn.LayerNorm(1000), nn.Linear(1000,1),nn.Sigmoid() )
    def forward(self, seq):
        n = seq.shape[1]
        batch_size = seq.shape[0]
        seqb = torch.flip(seq, [1])
        yf = []
        yb = []
        yf.append(nn.Sigmoid()(self.normf(self.current_itemf(seq[:,0,:]))))
        yb.append(nn.Sigmoid()(self.normb(self.current_itemb(seqb[:,0,:]))))
        yf.append(nn.Sigmoid()(self.normf(self.current_itemf(seq[:,1,:])+ self.prev_itemf1(yf[-1]))))
        yb.append(nn.Sigmoid()(self.normb(self.current_itemb(seqb[:,1,:])+ self.prev_itemb1(yb[-1]))))
        yf.append(nn.Sigmoid()(self.normf(self.current_itemf(seq[:,2,:]) + self.prev_itemf1(yf[-1])+ self.prev_itemf2(yf[-2]))))
        yb.append(nn.Sigmoid()(self.normb(self.current_itemb(seqb[:,2,:]) + self.prev_itemb1(yb[-1])+ self.prev_itemb2(yf[-2]))))
        for i in range(3,n):
            # print(yf[-1].shape)
            yf.append(nn.Sigmoid()(self.normf(self.current_itemf(seq[:,i,:]) + self.prev_itemf1(yf[-1])+ self.prev_itemf2(yf[-2])+self.prev_itemf3(yf[-3]))))
            yb.append(nn.Sigmoid()(self.normb(self.current_itemb(seqb[:,i,:]) + self.prev_itemb1(yb[-1])+ self.prev_itemb2(yf[-2])+self.prev_itemb3(yf[-3]))))
        yb.reverse()

        changepoints = torch.zeros((batch_size, n))
        
        for i in range(n):
            
            changepoints[:,i] = self.point_detector(torch.cat((yf[i], yb[i]), 1))[:,0]

        return changepoints

In [4]:
class changeDetectLSTM(nn.Module):
    def __init__(self):
        super(changeDetectLSTM, self).__init__()
        self.lstm1 = nn.RNN(10, 10,1,batch_first=True, bidirectional=True)
        #  nn.LSTM(10, 20,5,batch_first=True, bidirectional=True)
        self.lstm2 = nn.LSTM(10, 5,3,batch_first=True)

        self.classifyLSTM = nn.LSTM(100, 1, 1, batch_first=True)
        self.classifier = nn.Sequential(nn.Linear(20,200), nn.ELU(), nn.LayerNorm(200),
                                        nn.Linear(200,100), nn.ELU(), nn.LayerNorm(100), nn.Linear(100,1))
        self.point_detector = nn.Sequential(nn.Linear(200,1000), nn.LayerNorm(1000),nn.Sigmoid(),nn.Linear(1000,1000),nn.Sigmoid(), nn.LayerNorm(1000), nn.Linear(1000,1),nn.Sigmoid() )

    
    def forward(self, seq):
        # seq = torch.mul(seq, 10)
        seq, _ = self.lstm1(seq)
        seq = self.classifier(seq)
        
        return seq

In [16]:
x = torch.randn(10)
print(x)
print(torch.cat((x,x), 0))

tensor([ 1.4984, -0.3807, -0.4210,  0.1526,  0.1696,  0.4242,  0.7521,  0.1741,
         0.8790,  0.4640])
tensor([ 1.4984, -0.3807, -0.4210,  0.1526,  0.1696,  0.4242,  0.7521,  0.1741,
         0.8790,  0.4640,  1.4984, -0.3807, -0.4210,  0.1526,  0.1696,  0.4242,
         0.7521,  0.1741,  0.8790,  0.4640])


In [5]:
def get_batch(batch_size, seq_length, device):
    x_batch = torch.zeros((batch_size, seq_length, 10), device=device)
    y_batch = torch.zeros((batch_size, seq_length), device=device)
    for i in range(batch_size):
        seq, _, _, haspoint, T = seqGen(seq_length)
        for j in range(len(seq)):
            if haspoint == 1 and j >= T:
                y_batch[i][j] = 1
            
        x_batch[i] = seq
    return x_batch, y_batch

In [67]:
tensor, _ = get_batch(1,10)
print(tensor)
print(torch.flip(tensor, [1]))

tensor([[[-1.4049, -0.8837,  0.2189,  0.2833,  0.0850, -1.1273, -0.5754,
           0.3827,  0.4344, -0.8288],
         [-1.0201, -0.6424, -0.3385, -0.1234, -0.4842, -1.3048, -0.5308,
           1.0170,  0.7136,  1.6644],
         [ 1.3211,  0.5331, -1.3790,  0.1452, -0.2916,  0.6121, -0.3462,
          -1.4847,  1.1317, -0.2574],
         [-2.4638, -1.2155, -1.1739, -1.5042,  1.0520,  0.7329,  0.4771,
          -0.0115,  0.6189, -0.6080],
         [-0.5209,  0.4390, -0.3737,  1.7966, -0.9788,  0.2802, -1.2945,
          -1.8794, -1.3292, -0.3854],
         [-0.3010,  1.3106, -0.8410, -1.0394, -0.6550,  0.9290,  0.7821,
           0.5405,  0.3706,  0.8595],
         [-0.7153,  1.0079, -0.0314, -1.0289, -2.5729, -0.9301,  1.1003,
           1.5781, -0.8748, -1.3504],
         [ 0.2485, -0.7091,  0.9974,  2.6215, -0.8839, -1.7273,  0.4768,
           0.5396, -0.7275,  0.0954],
         [ 0.5676,  0.8866, -0.3271,  1.1786,  0.2498, -1.3037, -1.3073,
          -0.7880,  1.5421,  1.2406],
 

In [11]:
def lossfunc(predictions, actual_labels):
    return torch.mean( -1 * torch.log( predictions + 0.0001 ).to(device) * actual_labels - torch.log( 1 - predictions + 0.0001 ) * (1 - actual_labels ) )


In [7]:
model = changeDetectLSTM()
model = model.to(device)
optimizer = optim.Adam(model.parameters(), lr = 0.1)
x_test, y_test = get_batch(1000,100, device)
for epoch in range(400):
    totalLoss = 0
    for i in range(2**6):
        L = random.randint(2,100)
        # L = 10
        x_batch, y_batch = get_batch(64, L, device)
        optimizer.zero_grad 
        # print(y_batch.shape)
        output = model.forward(x_batch)
        # print(output.shape)
        # print(y_batch[0])
        loss = torch.nn.CrossEntropyLoss()(output[:,:,0], y_batch)
        totalLoss += loss.item()
        loss.backward()
        optimizer.step()
    print((epoch+1),"Test loss",torch.nn.CrossEntropyLoss(reduction='mean')(model.forward(x_test)[:,:,0], y_test).item(), "Training loss", totalLoss)



1 Test loss 116.81209564208984 Training loss 3507.976368010044
2 Test loss 117.92870330810547 Training loss 3729.502596616745
3 Test loss 119.26509857177734 Training loss 3715.859140753746
4 Test loss 118.62266540527344 Training loss 3206.1830335259438
5 Test loss 120.1336441040039 Training loss 3405.397356033325
6 Test loss 117.21597290039062 Training loss 3734.0032049417496
7 Test loss 117.03057098388672 Training loss 3357.841537952423
8 Test loss 117.29861450195312 Training loss 3262.6579213142395
9 Test loss 116.7618637084961 Training loss 3275.1898350715637
10 Test loss 116.6441650390625 Training loss 3251.3819422721863
11 Test loss 117.50759887695312 Training loss 3796.944023370743
12 Test loss 122.03592681884766 Training loss 3127.3643123209476
13 Test loss 127.79537963867188 Training loss 3614.823840856552
14 Test loss 117.5074462890625 Training loss 3441.292931675911
15 Test loss 117.22074127197266 Training loss 3529.906070590019
16 Test loss 116.75135040283203 Training loss 3

In [405]:

x,y = get_batch(1, 100, device)
print(y)
print(nn.Sigmoid()(model(x)[:,:,0]))

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.]], device='cuda:0')
tensor([[0.4496, 0.4573, 0.4482, 0.4603, 0.4501, 0.4457, 0.4735, 0.4493, 0.4499,
         0.4462, 0.4667, 0.4605, 0.4568, 0.4503, 0.4508, 0.4518, 0.4530, 0.4488,
         0.4455, 0.4598, 0.4500, 0.4536, 0.4507, 0.4546, 0.4557, 0.4551, 0.4547,
         0.4457, 0.4531, 0.4715, 0.4490, 0.4602, 0.4655, 0.4741, 0.4515, 0.4493,
         0.4499, 0.4487, 0.4497, 0.4657, 0.4456, 0.4536, 0.4477, 0.4480, 0.4515,
         0.4512, 0.4563, 0.4589, 0.4612, 0.4540, 0.4516, 0.4630, 0.4500, 0.4604,
         0.4485, 0.4584, 0.4536, 0.4673, 0.4506, 0

In [292]:
myRNN= nn.RNN(10,100,5).to(device)
print(myRNN(x))

(tensor([[[-0.0241, -0.0050, -0.0436,  ...,  0.0630, -0.1782,  0.0905],
         [-0.0416, -0.0136, -0.0568,  ...,  0.0361, -0.1718,  0.0904],
         [-0.0542, -0.0191, -0.0371,  ...,  0.0546, -0.1764,  0.0810],
         ...,
         [-0.0719, -0.0675, -0.0368,  ...,  0.0970, -0.2168,  0.0861],
         [-0.0404, -0.0170, -0.0637,  ...,  0.0535, -0.1707,  0.0789],
         [-0.0438, -0.0119, -0.0418,  ...,  0.0529, -0.1878,  0.0725]]],
       device='cuda:0', grad_fn=<CudnnRnnBackward0>), tensor([[[ 0.3606,  0.3763, -0.0703,  ...,  0.4405, -0.0639, -0.0504],
         [-0.1602,  0.0735,  0.1989,  ..., -0.1785, -0.3362,  0.3406],
         [ 0.3833,  0.1238,  0.0771,  ..., -0.2267,  0.0039,  0.1872],
         ...,
         [ 0.0477, -0.1614,  0.4257,  ..., -0.1218, -0.0999, -0.2118],
         [ 0.2183,  0.1890,  0.2391,  ..., -0.0024, -0.1767,  0.2191],
         [ 0.3539,  0.1075, -0.1330,  ...,  0.2023,  0.1865,  0.1567]],

        [[ 0.1841, -0.1916,  0.1030,  ..., -0.1199, -0.0565, 